## Initialise Classes and Functions

In [123]:
class Node:
    def __init__(self, val):
        self.val = val # Value of node = its position (left, right, zigzag, continue).
        self.adj_list = set()

# create a list of edges for Game
class Graph:
    def __init__(self, num_rows, num_cols):
        self.num_rows = num_rows
        self.num_cols = num_cols
        self.nodes_int = self._create_nodes()
        
        self.matrix = [[0] * 13 for _ in range(13)]

        
        self.edges = set()
        for row in range(self.num_rows): # Kinda like initialising a 2D matrix. From the nodes generated, add the edges to the set.
            for col in range(self.num_cols):
                node = self.nodes_mat[row][col]
                
                adj_list = self.mapper[node.val].adj_list
                node = self.mapper[node.val]
                for other_node in adj_list:
                    self.edges.add((node, other_node))
                    self.edges.add((other_node, node))
        
    def init_start_and_end(self):
        self.start = 1 # top-left
        self.end = self.num_rows * self.num_cols # bottom-right
        
    def _create_nodes(self):
        nodes_int, i = [], 1
        nodes = [[Node((row, col)) for col in range(self.num_cols)] for row in range(self.num_rows)] 
        mapper = dict()
        int_mapper = dict()
        for row in range(self.num_rows):
            for col in range(self.num_cols):
                node = Node(i) # Create the nodes, number 1 to (m * n).
                nodes_int.append(node)
                mapper[(row, col)] = node
                int_mapper[i] = node
                i += 1
        
        for row in range(self.num_rows):
            for col in range(self.num_cols):
                node = nodes[row][col] # this stuff below is where we initialise the neighbours.
                if row > 0:
                    node.adj_list.add(nodes[row - 1][col])  # Upper neighbour.
                    nodes[row - 1][col].adj_list.add(node)
                    
                    current_mapping = mapper[(row, col)]
                    nbr_mapping = mapper[(row - 1, col)]
                    
                    current_mapping.adj_list.add(nbr_mapping)
                    nbr_mapping.adj_list.add(current_mapping)
                    
                if row < self.num_rows - 1:
                    node.adj_list.add(nodes[row + 1][col])  # Lower neighbour.
                    nodes[row + 1][col].adj_list.add(node)
                    
                    current_mapping = mapper[(row, col)]
                    nbr_mapping = mapper[(row + 1, col)]
                    
                    current_mapping.adj_list.add(nbr_mapping)
                    nbr_mapping.adj_list.add(current_mapping)
                    
                if col > 0:
                    node.adj_list.add(nodes[row][col - 1])  # Left neighbour.
                    nodes[row][col - 1].adj_list.add(node)
                    
                    current_mapping = mapper[(row, col)]
                    nbr_mapping = mapper[(row, col - 1)]
                    
                    current_mapping.adj_list.add(nbr_mapping)
                    nbr_mapping.adj_list.add(current_mapping)
                    
                if col < self.num_cols - 1:
                    node.adj_list.add(nodes[row][col + 1])  # Right neighbour.
                    nodes[row][col + 1].adj_list.add(node)
                    
                    current_mapping = mapper[(row, col)]
                    nbr_mapping = mapper[(row, col + 1)]
                    
                    current_mapping.adj_list.add(nbr_mapping)
                    nbr_mapping.adj_list.add(current_mapping)
                
        self.nodes_mat = nodes
        self.mapper = mapper
        self.int_mapper = int_mapper
                    
        return nodes_int
        
    def print_graph(self): # for debugging purposes
        print([node.val for row in self.nodes_mat for node in row])
        print()
        print([node.val for node in self.nodes_int])

In [124]:
import random
import torch
from collections import deque
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR

def format_state(state):
    num_nodes = find_max_number([state[0], state[1], state[2]])
        
    secured_edges = convert_to_adj_matrix(state[0], num_nodes)
    deleted_edges = convert_to_adj_matrix(state[1], num_nodes)
    remaining_edges = convert_to_adj_matrix(state[2], num_nodes)
    secured_count, remaining_count = state[3], state[4]
        
    secured_edges, deleted_edges, remaining_edges, secured_count, remaining_count = (torch.tensor(secured_edges).flatten(), 
                                                                                     torch.tensor(deleted_edges).flatten(), 
                                                                                     torch.tensor(remaining_edges).flatten(), 
                                                                                     torch.tensor([secured_count]), 
                                                                                     torch.tensor([remaining_count]))
            
    formatted_state = np.concatenate([secured_edges, deleted_edges, remaining_edges, secured_count, remaining_count]).tolist()
    formatted_state = torch.tensor(formatted_state)
    
    return formatted_state

def convert_to_adj_matrix(edges, num_nodes):
    nodes = set()
    for edge in edges:
        nodes.add(edge[0])
        nodes.add(edge[1])

    adj_matrix = np.zeros((num_nodes + 1, num_nodes + 1)) # 0th col and 0th row will just be to pad.

    # Populate the adjacency matrix
    for edge in edges:
        adj_matrix[edge[0]][edge[1]] = 1
        
    return adj_matrix

def find_max_number(lists_of_tuples):
    max_number = float('-inf')

    for list_of_tuples in lists_of_tuples:
        for tup in list_of_tuples:
            numbers = [x for x in tup if isinstance(x, (int, float))]
            if numbers:
                current_max = max(numbers)
                if current_max > max_number:
                    max_number = current_max

    return max_number

In [125]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import os

# This is the Feedforward Neural Network.
class ShannonModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, edges, game):
        self.edges = edges
        self.game = game
        
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        
        sorted_x, indices = torch.sort(x, descending = True)
        max_probability = None
        chosen_edge = None
    
        for index in indices:
            edge = self.edges[index.item()]
            reverse_edge = (edge[1], edge[0])
            
            if ((edge not in self.game.removed_edges and edge not in self.game.secured_edges) and 
                (reverse_edge not in self.game.removed_edges and reverse_edge not in self.game.secured_edges)): 
                max_probability = x[index]
                chosen_edge = edge
                break
                
        return x, max_probability, chosen_edge # Return the probabilities and the valid edge with the max probability.

In [131]:
# a couple coniditons (0-based indexing)
# 1. even number row, even number col: put a NUMBER ([1-49])
# 2. even number row, odd number col: put a DASH (-)
# 3. odd number row, even number col: put a DASH (|)
# 4. odd number row, odd number col: put an empty SPACE ()

counter = 1
boner = [[0] * 13 for _ in range(13)]

edge_mapping = dict() # tuples of nodes (e.g. (1, 2))
for row in range(len(boner)):
    for col in range(len(boner[row])):
        if row % 2 == 0:
            if col % 2 == 0: # 1.
                boner[row][col] = counter
                counter += 1

for row in range(len(boner)):
    for col in range(len(boner[row])):
        if row % 2 == 0:
            if col % 2 != 0: # 2.
                boner[row][col] = "-" # edge
                
                # look to the left and right
                left = boner[row][col - 1]
                right = boner[row][col + 1]
                edge_mapping[(left, right)] = (row, col)
        else:
            if col % 2 == 0: # 3
                boner[row][col] = "|"
                
                # look up and down
                up = boner[row - 1][col]
                down = boner[row + 1][col]
                edge_mapping[(up, down)] = (row, col)
            else:
                boner[row][col] = ""

In [139]:
import random
class GameAI:
    def __init__(self, graph):
        self.graph = graph
        self.m = self.graph.num_rows
        self.n = self.graph.num_cols
        
        self.node_mapping = dict()
        for i in range(1, (self.m * self.n) + 1):
            self.node_mapping[i] = self.graph.nodes_int[i - 1] # e.g. 1 is in index 0, 2 is index 1, etc.
        
        self.edges = []
        for i in range(1, (self.graph.num_rows * self.graph.num_cols) + 1):
            adj_list = [node.val for node in self.graph.int_mapper[i].adj_list]
            for adj_node in adj_list:
                if (adj_node, i) in self.edges:
                    continue
                self.edges.append((i, adj_node))
        
        self.edges = sorted(self.edges)
        
        self.unsecured_count = (2 * self.m * self.n) - self.m - self.n # this is for the CUT player
        self.secured_edges = [] # fix  
        self.removed_edges = [] # cut
        
        self.remaining = {(node1.val, node2.val) for node1, node2 in self.graph.edges} 
        self.cut_win = False
        self.end = False
        
        self.matrix = [[0] * 13 for _ in range(13)]
        self.init_matrix()
        
    def reset(self): # Reset everything for the next training iteration.
        self.unsecured_count = (2 * self.m * self.n) - self.m - self.n 
        self.secured_edges = []
        self.removed_edges = []
        
        self.remaining = {(node1.val, node2.val) for node1, node2 in self.graph.edges} 
        self.cut_win = False
        self.end = False
        
        self.matrix = [[0] * 13 for _ in range(13)]
        self.init_matrix()
        
    def init_matrix(self):
        self.edge_mapping = dict() # tuples of nodes (e.g. (1, 2))
        counter = 1
        for row in range(len(boner)):
            for col in range(len(boner[row])):
                if row % 2 == 0:
                    if col % 2 == 0: # 1.
                        self.matrix[row][col] = counter
                        counter += 1

        for row in range(len(boner)):
            for col in range(len(boner[row])):
                if row % 2 == 0:
                    if col % 2 != 0: # 2.
                        # self.matrix[row][col] = "-" # edge
                        self.matrix[row][col] = "" # edge
                
                        # look to the left and right
                        left = self.matrix[row][col - 1]
                        right = self.matrix[row][col + 1]
                        self.edge_mapping[(left, right)] = (row, col)
                else:
                    if col % 2 == 0: # 3
                        # self.matrix[row][col] = "|"
                        self.matrix[row][col] = ""
                
                        # look up and down
                        up = self.matrix[row - 1][col]
                        down = self.matrix[row + 1][col]
                        self.edge_mapping[(up, down)] = (row, col)
                    else:
                        self.matrix[row][col] = ""
        
    # Random now mate.
    def choose_edge_to_fix(self):
        edge_to_fix = random.choice(list(self.remaining))
        return edge_to_fix
    
    # Plays step-by-step. This is what we'll use for "learning".
    def next_step_player(self, chosen_edge, user_edge):
        if self.unsecured_count > 0 and not self.end:
            if not self.end:
                # 1. CUT player's turn: where the magic happens. (HE NOW GOES FIRST)
                if len(self.remaining) == 0:
                    # No more valid edges to choose.
                    print("You lost mate.")
                    self.cut_win = True
                    self.end = True
                else:
                    self.cut(chosen_edge)
                    self.update_board(chosen_edge, cut = True)
            
            # 2. FIX/user player's turn.
            if not self.end:
                if len(self.remaining) == 0:
                    # No more valid edges to choose.
                    print("You lost mate.")
                    self.cut_win = True
                    self.end = True
                else:
                    edge_to_fix = user_edge
                    self.fix(edge_to_fix)
                    self.update_board(edge_to_fix, cut = False)
            
            self.print_board()
                    
            if self.is_fix_path_complete():
                print("You won!")
                self.cut_win = False
                self.end = True
                return      
            elif len(self.remaining) == 0:
                print("You lost mate.")
                self.cut_win = True
                self.end = True
        else:
            print("You lost mate.")
            self.end = True
            self.cut_win = True
            
    def update_board(self, edge, cut):
        coordinates = self.edge_mapping[edge]
        row, col = coordinates[0], coordinates[1]
        
        if cut:
            self.matrix[row][col] = "X"
        else: # if it's the user's turn
            # if vertical
            if row % 2 != 0:
                self.matrix[row][col] = "|"
            else: # if horiz
                self.matrix[row][col] = "-"

    def is_fix_path_complete(self): # This does BFS to check if there is a path from the start to the end.
        visited = set()
        stack = [1]

        while stack:
            current_node = stack.pop()
            if current_node == self.m * self.n: # e.g. 4x4, 16 is the bottom-right.
                return True

            if current_node not in visited:
                visited.add(current_node)
                adj_list = self.node_mapping[current_node].adj_list
                
                for nbr in adj_list:
                    edge = (current_node, nbr.val)
                    reverse_edge = (nbr.val, current_node)

                    if edge in self.secured_edges or reverse_edge in self.secured_edges:
                        if nbr.val not in visited:
                            stack.append(nbr.val)

        return False
    
    # 1. FIX player's function; secures unsecured edge in question (and its reverse).
    def fix(self, edge):
        # edge = ex: (1, 4)
        if edge in self.remaining:
            self.remaining.remove(edge)
            self.secured_edges.append(edge)
            self.unsecured_count -= 1
            
        # Also remove the reverse direction of the edge.
        reverse_edge = (edge[1], edge[0])
        if reverse_edge in self.remaining:
            self.remaining.remove(reverse_edge)

    # 2. CUT player's function; removes unsecured edge in question (and its reverse).
    def cut(self, edge):
        # edge = ex: (1, 4)
        if edge in self.remaining:
            self.remaining.remove(edge)
            self.removed_edges.append(edge)
            self.unsecured_count -= 1
            
        # Also remove the reverse direction of the edge.
        reverse_edge = (edge[1], edge[0])
        if reverse_edge in self.remaining:
            self.remaining.remove(reverse_edge)
  
    # Reward function
    def get_reward(self):
        if self.cut_win:
            # Positive reward when the CUT player wins
            reward = 1.0
        elif self.end:
            # Negative reward when the CUT player loses
            reward = -1.0
        else:
            # Intermediate "reward" for the ongoing game
            reward = 0.1            

        return reward
            
    def get_state(self):
        # Define the state representation based on the game state.
        secured_count = len(self.secured_edges)
        remaining_count = self.unsecured_count
        secured_edges = self.secured_edges
        deleted_edges = self.removed_edges
        remaining_edges = list(self.remaining) # Yet for this, we'll keep the reverse edges. Bit hypocritical, but fuck it.
        
        state = (secured_edges, deleted_edges, remaining_edges, secured_count, remaining_count)
    
        return state
    
    def print_board(self):
        for row in self.matrix:
            for element in row:
                print(f"{element:>{max_width}}", end=" ")
            print()
        pass
    
    # This is still here purely for debugging purposes.
    def play(self):
        while self.unsecured_count > 0:
            # 1. CUT player's turn
            if len(self.remaining) == 0:
                # No more valid edges to choose.
                self.cut_win = True
                self.end = True
                break
                
            edge_to_cut = self.choose_edge_to_cut()
            self.cut(edge_to_cut)
        
            # 2. FIX player's turn
            if len(self.remaining) == 0:
                # No more valid edges to choose.
                self.cut_win = True
                self.end = True

            edge_to_fix = self.choose_edge_to_fix()
            self.fix(edge_to_fix)
            
            if self.is_fix_path_complete():
                self.cut_win = False
                self.end = True
                break
            elif self.unsecured_count == 0:
                self.cut_win = True
                self.end = True
                
    # This is still here purely for debugging purposes.
    def choose_edge_to_cut(self):
        edge_to_cut = random.choice(list(self.remaining))
        return edge_to_cut

## Read in Model

In [15]:
num_rows = 7
num_cols = 7
graph = Graph(num_rows, num_cols)
game = GameAI(graph)
edges = game.edges

cut_ai = ShannonModel(input_size = 7502, hidden_size = 256, output_size = 84, edges = edges, game = game)

In [16]:
cut_ai.load_state_dict(torch.load("Cut/7x7Model.pt"))

<All keys matched successfully>

## Sanity Check Model 

In [20]:
cut_ai.eval()
train_wins = 0
for i in range(1000):
    if i % 100 == 0:
        print(1000 - i, "iterations remaining.")
    
    game.reset()
    while not game.end:
        state = format_state(game.get_state())
        _, _, chosen_edge = cut_ai(state)
        game.next_step_player(chosen_edge)
        
    if game.cut_win:
        train_wins += 1

1000 iterations remaining.
900 iterations remaining.
800 iterations remaining.
700 iterations remaining.
600 iterations remaining.
500 iterations remaining.
400 iterations remaining.
300 iterations remaining.
200 iterations remaining.
100 iterations remaining.


In [23]:
game.reset()
train_wins

995

Only 5 losses. Good. It loaded properly. Get in. Ha'way the lads.

## Play Against It Chap

In [140]:
num_rows = 7
num_cols = 7
graph = Graph(num_rows, num_cols)
game = GameAI(graph)
edges = game.edges

cut_ai = ShannonModel(input_size = 7502, hidden_size = 256, output_size = 84, edges = edges, game = game)
cut_ai.load_state_dict(torch.load("Cut/7x7Model.pt"))
cut_ai.eval()

ShannonModel(
  (fc1): Linear(in_features=7502, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=84, bias=True)
)

In [146]:
import re
valid_edges = set(game.edges)
game.reset()

print("You are the fix player. You go 2nd, because why not.")
print("Input your numbers in the form of '(num1, num2)'")
while not game.end:
    state = format_state(game.get_state())
    _, _, chosen_edge = cut_ai(state)
    edge = ()
    
    print("AI has chosen to delete", chosen_edge)
    while True:
        edge = input("Secure next edge mate:")
        
        if re.match(r'^\(([1-9]|[1-3][0-9]|4[0-9]),([1-9]|[1-3][0-9]|4[0-9])\)$', edge):
            num1 = int(edge.replace("(", "").replace(")", "").split(",")[0])
            num2 = int(edge.replace("(", "").replace(")", "").split(",")[1])
            
            tup = (num1, num2)
            if tup in edges:
                edge = tup
                break
                
        print("Wrong format/improper edge. Try again.")
        
    game.next_step_player(chosen_edge, user_edge = edge)

You are the fix player. You go 2nd, because why not.
Input your numbers in the form of '(num1, num2)'
AI has chosen to delete (23, 30)


Secure next edge mate: (1,2)


 1  -  2     3     4     5     6     7 
                                       
 8     9    10    11    12    13    14 
                                       
15    16    17    18    19    20    21 
                                       
22    23    24    25    26    27    28 
       X                               
29    30    31    32    33    34    35 
                                       
36    37    38    39    40    41    42 
                                       
43    44    45    46    47    48    49 
AI has chosen to delete (19, 20)


Secure next edge mate: (2,3)


 1  -  2  -  3     4     5     6     7 
                                       
 8     9    10    11    12    13    14 
                                       
15    16    17    18    19  X 20    21 
                                       
22    23    24    25    26    27    28 
       X                               
29    30    31    32    33    34    35 
                                       
36    37    38    39    40    41    42 
                                       
43    44    45    46    47    48    49 
AI has chosen to delete (22, 23)


...

 1  -  2  -  3  -  4  -  5  -  6  -  7 
                                     | 
 8     9    10    11    12    13    14 
                                     | 
15    16    17    18  X 19  X 20    21 
                         X           | 
22  X 23    24  X 25  X 26  X 27    28 
 X     X           X                 | 
29    30    31    32    33    34    35 
                                       
36    37    38    39    40    41    42 
                                       
43    44    45    46    47    48    49 
AI has chosen to delete (17, 24)


Secure next edge mate: (35,42)


 1  -  2  -  3  -  4  -  5  -  6  -  7 
                                     | 
 8     9    10    11    12    13    14 
                                     | 
15    16    17    18  X 19  X 20    21 
             X           X           | 
22  X 23    24  X 25  X 26  X 27    28 
 X     X           X                 | 
29    30    31    32    33    34    35 
                                     | 
36    37    38    39    40    41    42 
                                       
43    44    45    46    47    48    49 
AI has chosen to delete (15, 22)


Secure next edge mate: (42,49)


 1  -  2  -  3  -  4  -  5  -  6  -  7 
                                     | 
 8     9    10    11    12    13    14 
                                     | 
15    16    17    18  X 19  X 20    21 
 X           X           X           | 
22  X 23    24  X 25  X 26  X 27    28 
 X     X           X                 | 
29    30    31    32    33    34    35 
                                     | 
36    37    38    39    40    41    42 
                                     | 
43    44    45    46    47    48    49 
You won!


Unfortunately, big limitation here. A bit pants. Needs more data.

## Two AIs Battling

In [147]:
num_rows = 7
num_cols = 7
graph = Graph(num_rows, num_cols)
game = GameAI(graph)
edges = game.edges

cut_ai = ShannonModel(input_size = 7502, hidden_size = 256, output_size = 84, edges = edges, game = game)
cut_ai.load_state_dict(torch.load("Cut/7x7Model.pt"))
cut_ai.eval()

fix_ai = ShannonModel(input_size = 7502, hidden_size = 256, output_size = 84, edges = edges, game = game)
fix_ai.load_state_dict(torch.load("Fix/7x7Model.pt"))
fix_ai.eval()

ShannonModel(
  (fc1): Linear(in_features=7502, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=84, bias=True)
)

In [149]:
import re
valid_edges = set(game.edges)
game.reset()

while not game.end:
    state = format_state(game.get_state())
    _, _, cut_chosen_edge = cut_ai(state)
    edge = ()
    
    print("Cut has chosen to delete", chosen_edge)
    
    _, _, fix_chosen_edge = fix_ai(state)
    edge = ()
    
    game.next_step_player(chosen_edge = cut_chosen_edge, user_edge = fix_chosen_edge)

Cut has chosen to delete (15, 22)
 1     2     3     4     5     6     7 
                                       
 8     9    10    11    12    13    14 
                                       
15    16    17    18    19    20    21 
                                       
22    23    24    25    26    27    28 
       X                               
29    30    31    32    33    34    35 
                                       
36    37    38    39    40    41    42 
                                       
43  - 44    45    46    47    48    49 
Cut has chosen to delete (15, 22)
 1     2     3     4     5     6     7 
                                       
 8     9    10    11    12    13    14 
                                       
15    16    17    18    19  X 20    21 
                                       
22    23    24    25    26    27    28 
       X                       |       
29    30    31    32    33    34    35 
                                       
36    37    

Three words sum this up well: Dumb and Dumber.